In [1]:
# import ee
# import geemap
# import pandas as pd
# from ipyleaflet import Popup, Marker

# # Initialize Earth Engine
# ee.Initialize()

# # Set display options to show all columns
# pd.set_option('display.max_columns', None)

# # Define the central point and buffer
# initial_point = ee.Geometry.Point(170.89167617626143, -43.99906841864817)
# roi = initial_point.buffer(40)  # 30-meter buffer around the central point

# # Setup image collection
# images = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2017-01-01', '2024-06-25').filterBounds(roi)

# # Get the first image to simplify the process
# image = images.first().select('B2')  # Select one band to keep it simple

# # Use sample to extract pixel data within the ROI
# scale = 10  # Sentinel-2 spatial resolution is 10 meters
# pixel_data = image.sample(region=roi, scale=scale, geometries=True).getInfo()

# # Extract coordinates from the sampled pixels
# coordinates = [(feature['geometry']['coordinates'][0], feature['geometry']['coordinates'][1]) for feature in pixel_data['features']]

# # Ensure unique coordinates
# unique_coords = list(set(coordinates))

# # Debugging: Print coordinates to verify
# print("Unique Coordinates:")
# for coord in unique_coords:
#     print(coord)

# # Visualise using geemap
# Map = geemap.Map(center=[-43.99906841864817, 170.89167617626143], zoom=15)
# s2_Image = images.sort('CLOUDY_PIXEL_PERCENTAGE').first()

# # Define visualization parameters
# vis_params = {
#     'bands': ['B4', 'B3', 'B2'],  # RGB bands
#     'min': 0,
#     'max': 3000,
#     'gamma': 1.4
# }

# # Add the Sentinel-2 image to the map
# Map.addLayer(s2_Image, vis_params, 'Sentinel-2')

# # Create a feature collection from the unique coordinates
# points = ee.Geometry.MultiPoint(unique_coords)

# # Define point visualization parameters
# point_params = {'color': 'FF0000'}  # Red color

# # Add points to the map interactively with info
# for coord in unique_coords:
#     marker = Marker(location=(coord[1], coord[0]), icon_color='red')
#     popup = Popup(location=(coord[1], coord[0]), child=geemap.widgets.HTML(value=f'Coordinates: {coord}'))
#     marker.popup = popup
#     Map.add(marker)

# # Display the map
# Map


# Using get region method.

In [2]:
import ee
import geemap
import pandas as pd
from ipyleaflet import Popup, Marker

# Initialize Earth Engine
ee.Initialize(project='data690-zhouhaomatt')

# Set display options to show all columns
pd.set_option('display.max_columns', None)

# Define the central point and buffer
initial_point = ee.Geometry.Point(170.89167617626143, -43.99906841864817)
roi = initial_point.buffer(40)  # 30-meter buffer around the central point

# Setup image collection
images = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2017-01-01', '2024-06-25').filterBounds(roi)

# Get the first image to simplify the process
image = images.first() # Select one band to keep it simple

# Use sample to extract pixel data within the ROI
scale = 10  # Sentinel-2 spatial resolution is 10 meters

band_list = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12']

pixel_values = images.select(band_list).getRegion(roi, scale).getInfo()

def convert_df(data, timezone='Etc/GMT-12'):
    df = pd.DataFrame(data[1:], columns=data[0])
    df['timestamp'] = pd.to_datetime(df['time'], unit='ms').dt.tz_localize('UTC').dt.tz_convert(timezone)
    df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    return df

pixel_df = convert_df(pixel_values)

unique_coords_df = pixel_df[['longitude', 'latitude']].drop_duplicates()

# Convert DataFrame to list of tuples
unique_coords = [tuple(x) for x in unique_coords_df.to_numpy()]

# Debugging: Print coordinates to verify
print("Unique Coordinates:")
for coord in unique_coords:
    print(coord)

# Visualise using geemap
Map = geemap.Map(center=[-43.99906841864817, 170.89167617626143], zoom=15)
s2_Image = images.sort('CLOUDY_PIXEL_PERCENTAGE').first()

# Define visualization parameters
vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # RGB bands
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}

# Add the Sentinel-2 image to the map
Map.addLayer(s2_Image, vis_params, 'Sentinel-2')

# Create a feature collection from the unique coordinates
points = ee.Geometry.MultiPoint(unique_coords)

# Define point visualization parameters
point_params = {'color': 'FF0000'}  # Red color

# Add points to the map interactively with info
for coord in unique_coords:
    marker = Marker(location=(coord[1], coord[0]), draggable=False)
    popup = Popup(location=(coord[1], coord[0]), child=geemap.widgets.HTML(value=f'Coordinates: {coord}'))
    marker.popup = popup
    Map.add_layer(marker)

# Display the map
Map

Unique Coordinates:
(170.89141231635503, -43.99934786888154)
(170.89150214788344, -43.99934786888154)
(170.89159197941186, -43.99934786888154)
(170.89168181094027, -43.99934786888154)
(170.89177164246868, -43.99934786888154)
(170.8918614739971, -43.99934786888154)
(170.89195130552548, -43.99934786888154)
(170.89132248482662, -43.99925803735313)
(170.89141231635503, -43.99925803735313)
(170.89150214788344, -43.99925803735313)
(170.89159197941186, -43.99925803735313)
(170.89168181094027, -43.99925803735313)
(170.89177164246868, -43.99925803735313)
(170.8918614739971, -43.99925803735313)
(170.89195130552548, -43.99925803735313)
(170.8920411370539, -43.99925803735313)
(170.8912326532982, -43.99916820582472)
(170.89132248482662, -43.99916820582472)
(170.89141231635503, -43.99916820582472)
(170.89150214788344, -43.99916820582472)
(170.89159197941186, -43.99916820582472)
(170.89168181094027, -43.99916820582472)
(170.89177164246868, -43.99916820582472)
(170.8918614739971, -43.99916820582472)
(

Map(center=[-43.99906841864817, 170.89167617626143], controls=(WidgetControl(options=['position', 'transparent…

In [3]:
# (170.8912326532982, -43.99907837429631),
# (170.8912326532982, -43.9989885427679),
# (170.89132248482662, -43.998898711239484),
#  (170.89141231635503, -43.99880887971107),
# (170.89159197941186, -43.99871904818266),
# (170.89168181094027, -43.99871904818266),
# (170.89177164246868, -43.99880887971107),
# (170.8918614739971, -43.99880887971107),
# (170.89195130552548, -43.99880887971107),
#  (170.8920411370539, -43.998898711239484),
# (170.8921309685823, -43.9989885427679),
#  (170.8921309685823, -43.99907837429631),
# (170.89132248482662, -43.99907837429631),
# (170.89132248482662, -43.9989885427679),
#  (170.89141231635503, -43.998898711239484),
# (170.89150214788344, -43.99880887971107),
# (170.89159197941186, -43.99880887971107),
#  (170.89168181094027, -43.99880887971107),
# (170.8918614739971, -43.998898711239484),
# (170.89195130552548, -43.998898711239484),
# (170.8920411370539, -43.9989885427679),
# (170.8920411370539, -43.99907837429631).



# NEW DATA EXTRACTION FROM SENTINEL 2

In [7]:
import ee
import pandas as pd

# Initialize Earth Engine
ee.Initialize(project='data690-zhouhaomatt')
# Set display options to show all columns
pd.set_option('display.max_columns', None)

# Coordinates extracted from the Sentinel-2 image centroids
# target_points = [
#     (170.8912326532982, -43.99907837429631),
#     (170.8912326532982, -43.9989885427679),
#     (170.89132248482662, -43.998898711239484),
#     (170.89141231635503, -43.99880887971107),
#     (170.89159197941186, -43.99871904818266),
#     (170.89168181094027, -43.99871904818266),
#     (170.89177164246868, -43.99880887971107),
#     (170.8918614739971, -43.99880887971107),
#     (170.89195130552548, -43.99880887971107),
#     (170.8920411370539, -43.998898711239484),
#     (170.8921309685823, -43.9989885427679),
#     (170.8921309685823, -43.99907837429631),
#     (170.89132248482662, -43.99907837429631),
#     (170.89132248482662, -43.9989885427679),
#     (170.89141231635503, -43.998898711239484),
#     (170.89150214788344, -43.99880887971107),
#     (170.89159197941186, -43.99880887971107),
#     (170.89168181094027, -43.99880887971107),
#     (170.8918614739971, -43.998898711239484),
#     (170.89195130552548, -43.998898711239484),
#     (170.8920411370539, -43.9989885427679),
#     (170.8920411370539, -43.99907837429631)
# ]
target_points = [
    (170.89142642028511, -43.99924808088317),
    (170.89155129827327, -43.99924808088317),
    (170.89167617626143, -43.99924808088317),
    (170.89180105424958, -43.99924808088317),
    (170.89192593223774, -43.99924808088317),
    (170.89142642028511, -43.99915824976567),
    (170.89192593223774, -43.99915824976567),
    (170.89142642028511, -43.99906841864817),
    (170.89192593223774, -43.99906841864817)
]
# Create EE geometry points (server-side objects)
ee_points = [ee.Geometry.Point(lon, lat) for lon, lat in target_points]

# Define the ROI using a MultiPoint geometry (server-side object)
region = ee.Geometry.MultiPoint(ee_points)

# Setup image collection
images = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2017-01-01', '2024-06-25').filterBounds(region)
clouds = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED').filterDate('2017-01-01', '2024-06-25').filterBounds(region)

# Data extraction setup
scale = 10  # Spatial resolution in meters
band_list = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12']
cloud_bands = ['cs', 'cs_cdf']

# Fetch pixel and cloud values from Earth Engine and bring them to the client side
try:
    pixel_values = images.select(band_list).getRegion(region, scale).getInfo()
    cloud_values = clouds.select(cloud_bands).getRegion(region, scale).getInfo()
except Exception as e:
    print(f"Failed to retrieve data: {e}")
    exit()

# Convert the extracted data to pandas DataFrames and adjust timestamps
def convert_df(data, timezone='Etc/GMT-12'):
    df = pd.DataFrame(data[1:], columns=data[0])
    df['timestamp'] = pd.to_datetime(df['time'], unit='ms').dt.tz_localize('UTC').dt.tz_convert(timezone)
    df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    return df

pixel_values_df = convert_df(pixel_values)
cloud_values_df = convert_df(cloud_values)

# Function to extract sun angles from image metadata
def extract_sun_angles(image):
    # Extract the sun zenith and azimuth angles as ee.Number objects
    sun_zenith = ee.Number(image.get('MEAN_SOLAR_ZENITH_ANGLE'))
    sun_azimuth = ee.Number(image.get('MEAN_SOLAR_AZIMUTH_ANGLE'))
    # Return an ee.Feature with these properties for standardized handling
    return ee.Feature(None, {
        'system:time_start': image.get('system:time_start'), 
        'sun_zenith': sun_zenith, 
        'sun_azimuth': sun_azimuth
    })

# Apply the function to each image in the collection and fetch the data to the client side
sun_angles = images.map(extract_sun_angles).getInfo()

# Convert the sun angles information to a pandas DataFrame
sun_angles_df = pd.DataFrame([{
    'timestamp': pd.to_datetime(image['properties']['system:time_start'], unit='ms').tz_localize('UTC').tz_convert('Etc/GMT-12').strftime('%Y-%m-%d %H:%M:%S'), 
    'sun_zenith': image['properties']['sun_zenith'], 
    'sun_azimuth': image['properties']['sun_azimuth']
} for image in sun_angles['features']])

# Mapping points to identifiers
point_ids = {point: f"point{i+1}" for i, point in enumerate(target_points)}

# Function to pivot the data to have one row per timestamp with all pixel information
def pivot_dataframe(df, value_columns):
    df['point_id'] = df.apply(lambda row: point_ids.get((row['longitude'], row['latitude']), f"unknown_{row['longitude']}_{row['latitude']}"), axis=1)
    df_pivot = df.pivot(index='timestamp', columns='point_id', values=value_columns)
    df_pivot.columns = [f"{col[1]}_{col[0]}" for col in df_pivot.columns]
    df_pivot.reset_index(inplace=True)
    return df_pivot

# Pivot pixel values and cloud values dataframes
pixel_values_pivot = pivot_dataframe(pixel_values_df, band_list)
cloud_values_pivot = pivot_dataframe(cloud_values_df, cloud_bands)

# Merge pixel, cloud, and sun angles dataframes
def merge_dataframes(pixels_pivot, clouds_pivot, sun_df):
    merged_df = pd.merge(pixels_pivot, clouds_pivot, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, sun_df, on='timestamp', how='left')
    return merged_df

# Merge dataframes and print the final result
final_df = merge_dataframes(pixel_values_pivot, cloud_values_pivot, sun_angles_df)

# Display the final DataFrame
print(final_df)


                timestamp  unknown_170.89141231635503_-43.99907837429631_B2  \
0     2017-01-05 10:25:23                                               NaN   
1     2017-01-08 10:37:20                                               NaN   
2     2017-01-15 10:25:21                                               NaN   
3     2017-01-18 10:37:19                                               NaN   
4     2017-01-25 10:25:30                                               NaN   
...                   ...                                               ...   
1041  2024-06-15 10:38:25                                             392.0   
1042  2024-06-17 10:28:31                                            2946.0   
1043  2024-06-20 10:38:27                                            1350.0   
1044  2024-06-22 10:28:27                                            1144.0   
1045  2024-06-25 10:38:25                                            2314.0   

      unknown_170.89141231635503_-43.99916820582472

In [8]:
final_df = final_df.dropna()

In [9]:
final_df.to_csv('26_June_test_sentinel2.csv', index=False)